# Calculation of Surface Settlements above Tunnel

## Aim

Estimate surface settlements above shallow tunnels in soft ground.  

## Method

Estimates of surface settlements above tunnels can be made using empirical procedures. 
The approaches typically used are: 
* estimating lost ground during during tunnel excavation and equating this to the settlement trough
* calculating displacement of tunnel crown to estimate surface settlement at tunnel centerline.

The procedure used here uses the lost ground approach and assumes that the form of the settlement trough follows a Gauss curve (Peck 1969). This procedures requires two coefficeints:
* volume of lost ground
* distance from tunnel centerline to inflection point of Gauss curve.

These coefficients are typically estimated using: 
* empirical data based on the ground characteristics and tunnel excavation method
* statistical inference from similar tunnels
* backcalculation from measurements during tunnel excavation.

The estimation of coefficients using empirical data may consider stress analysis, such as the overload factor used in the procedure developed by Peck. More recent experience has however shown that site-specific geological and excavation factors dominate the actual settlement during tunnel excavation. 

The procedure implemented here uses estimates of the coefficients from comparable tunnel work, supplemented by statistical data and backcalculation data where available.

This procedure is valid for single tunnels and is not applicable to closely-spaced tunnels.

v0.5  2017.11.27  JK

In [1]:
# import python libraries
from numpy import *
import pandas as pd

In [2]:
# set up plotly in offline mode
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as plotly


## Input Data

<img src="SettlementTrough.png" width=50% align="left">

In [3]:
# basic tunnel data                                             #ToDo: revise figure to correspond to input naming
D = 13.0            #tunnel diameter (m)
A = pi*(D/2)**2     #tunnel excavation cross-section (m2)
d = 13.0            #distance between ground surface and tunnel centerline (m)

## Setlement Calculation

In [4]:
# document calculation procedures (equations ...) with Markdown/Latex
# http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html
# some examples                                                                 #ToDo: enter correct equations 

$e^{i\pi} + 1 = 0$
$$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$$

In [5]:
# empirical calculation of settlement trough over tunnel (Gauss curve approach)
#                                                                                                   WBS: 'PM'
#                                                                                                   WBS: 'Feature'
#                                                                              #ToDo: how to call from Tunnel GIS

# empirical estimates of calculation coefficients (coeff's must have save number of items)
# coeff's for calculation results 0,1
# these coeff's are for bored tunnels in soil
c_v = array([0.002, 0.006]) #empirical values for lost ground as fraction of excavation area
c_i = array([0.7, 0.3]) #empirical coeff's to estimate distance centerline to inflection point of gauss curve

# statistical estimates of calculation coefficients
# coeff's for calculation results 2,3
# nach Fillibeck für nichtbindigen Boden, dicht gelagert
c_i = append(c_i, (0.5, 0.5))  
c_v = append(c_v, (0.0033 * (A/d)**-0.82, 0.0064 * (A/d)**-0.75))  #50%, 90% Quantile
# coeff's for calculation results 4,5
# nach Fillibeck für bindigen Boden, dicht gelagert
c_i = append(c_i, (0.7, 0.7))  
c_v = append(c_v, (.0033 * (A/d)**-0.82, 0.0064 * (A/d)**-0.75))  #50%, 90% Quantile

# backcalculation estimates of calculation coefficients
# placeholder

# calculate intermediate results (d assumed constant along cross-section)
vol_loss = A * c_v
infl_len = d * c_i
# calculte settlement at centerline
setl_max = vol_loss / (sqrt(2*pi) * infl_len)
# calculate settlment at distance x along gauss curve                          #ToDo: lambda or function preferred?
setl = lambda x, setl_max, infl_len: (
    setl_max * exp(- x**2 / (2 * infl_len**2)) )

# calculate settlment at distance x along cross-section
x = arange(-30,31,1)
setl_rslts = []

for i in x:
    setl_rslts.append(setl(i, setl_max, infl_len))

## Calculation Results

In [6]:
# write intermediate results
print('lost volume (m3): '+str(vol_loss))
print('inflection length (m): '+str(infl_len))
print('max settlement (m): '+str(setl_max))

#x=10.4  #for checking
#print('settlement (m) at: '+str(x)+'m = '+str(
#        setl(x, setl_max, infl_len)))

lost volume (m3): [ 0.26546458  0.79639374  0.06517534  0.1487242   0.06517534  0.1487242 ]
inflection length (m): [ 9.1  3.9  6.5  6.5  9.1  9.1]
max settlement (m): [ 0.01163792  0.08146542  0.00400018  0.00912806  0.00285727  0.00652004]


In [7]:
# plot results
nr_rslts = len(setl_rslts[0])
y_data = [[] for n in range(nr_rslts)]
trace = [[] for n in range(nr_rslts)]
    
for n in range(nr_rslts):
    for i in setl_rslts:
        y_data[n].append(-i[n])
    trace[n] = go.Scatter(
        x=x,
        y=y_data[n],
        name='result '+str(n) )

data = go.Data(trace)
layout=go.Layout(title="Surface Settlement above Bored Tunnel", 
                 xaxis={'title':'distance from centerline'}, yaxis={'title':'depth'})
figure=go.Figure(data=data,layout=layout)
plotly.iplot(figure)

## Conclusions

Add discussion of results here.